# 数据增广

由于之前的0到3的小demo中最终得到的准确率结果都不好，前面测试中最好的模型只是0.62（AlexNet），所以这里考虑是数据量太少了的问题来重新进行数据增广，增加数据量。


## ask chatgpt

> pytorch CIFAR10任务，我有train_data（50000，3，32，32）,train_labels（50000）,test_data（10000，3，32，32）,test_labels（10000）四个tensor，你可以告诉我如何使用transform来进行数据增广吗，比如重塑宽高、翻转、张量化、规范化

当我们进行深度学习任务时，使用数据增强技术可以提高模型的泛化能力。在 PyTorch 中，我们可以使用 transforms 模块来对数据进行增强。下面是一个使用 transforms 对 CIFAR10 数据集进行数据增强的示例：

```python
import torch
import torchvision
import torchvision.transforms as transforms

# 定义 transform，包括随机水平翻转、随机裁剪、归一化
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 只需要归一化
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载 CIFAR10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

```

在上面的代码中，transform_train 定义了训练数据的增强方式，包括随机水平翻转、随机裁剪、张量化、规范化等步骤。transform_test 定义了测试数据的增强方式，只包括张量化和规范化。

通过 transforms.Compose 将多个 transform 组合起来，形成一个 transform 序列。在加载数据集时，将这个 transform 序列传递给 transform 参数即可实现数据增强。

使用 transforms.RandomCrop() 可以进行裁剪操作，使用 transforms.RandomHorizontalFlip() 可以进行随机水平翻转。对于张量化和规范化，使用 transforms.ToTensor() 将数据转换成 tensor 格式，并使用 transforms.Normalize() 将数据规范化到均值为 0、标准差为 1。

在上面的代码中，我们定义了训练数据和测试数据的 transform 不同，因为测试数据不需要进行数据增强，只需要进行归一化即可。

最后，使用 DataLoader 加载数据集时，可以设置 batch_size 和 shuffle 等参数，进行批处理和打乱数据的操作。


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# 定义 transform，包括随机水平翻转、随机裁剪、归一化
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 只需要归一化
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载 CIFAR10 数据集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## 定义模型

In [2]:
from models import AlexNet
from torchinfo import summary
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AlexNet()
model.to(device)
summary(model, input_size=(10, 3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [10, 10]                  --
├─Sequential: 1-1                        [10, 256, 1, 1]           --
│    └─Conv2d: 2-1                       [10, 64, 17, 17]          1,792
│    └─ReLU: 2-2                         [10, 64, 17, 17]          --
│    └─MaxPool2d: 2-3                    [10, 64, 8, 8]            --
│    └─Conv2d: 2-4                       [10, 192, 8, 8]           307,392
│    └─ReLU: 2-5                         [10, 192, 8, 8]           --
│    └─MaxPool2d: 2-6                    [10, 192, 3, 3]           --
│    └─Conv2d: 2-7                       [10, 384, 3, 3]           663,936
│    └─ReLU: 2-8                         [10, 384, 3, 3]           --
│    └─Conv2d: 2-9                       [10, 256, 3, 3]           884,992
│    └─ReLU: 2-10                        [10, 256, 3, 3]           --
│    └─Conv2d: 2-11                      [10, 256, 3, 3]           

## 开始训练

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from models import LeNet, AlexNet
from tqdm import tqdm
import os
import numpy as np


class ArgumentConfig:
    def __init__(self,
        lr=0.001,
        resume=False,
        model='AlexNet',
        epochs=3,
        batch_size=16,
        n_splits=5,
        is_print=True,
        print_every=1
        ):
        self.lr = lr
        self.resume = resume
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        self.n_splits = n_splits
        self.is_print = is_print
        self.print_every = print_every

args = ArgumentConfig()


In [4]:
# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 定义优化器
optimizer = optim.Adam(model.parameters(), lr=args.lr)

# 定义学习率衰减
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

# 定义 tensorboard
writer = SummaryWriter(log_dir='runs/' + args.model)

# 定义是否使用 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 定义是否加载模型
if args.resume:
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    model.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

# 定义训练函数
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        if batch_idx % args.print_every == 0 and args.is_print:
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAcc: {:.3f}%'.format(
                epoch, batch_idx * len(inputs), len(trainloader.dataset),
                       100. * batch_idx / len(trainloader), loss.item(),
                100. * correct / total))
    writer.add_scalar('Train/Loss', train_loss / (batch_idx + 1), epoch)
    writer.add_scalar('Train/Acc', 100. * correct / total, epoch)

# 定义测试函数
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    writer.add_scalar('Test/Loss', test_loss / (batch_idx + 1), epoch)
    writer.add_scalar('Test/Acc', 100. * correct / total, epoch)
    return 100. * correct / total


In [5]:
torch.__version__

'1.13.1+cpu'

In [6]:
for epoch in range(args.epochs):
    train(epoch)
    acc = test(epoch)
    scheduler.step()
    # 保存模型
    state = {
        'net': model.state_dict(),
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/ckpt.pth')

Epoch: 0 [0/50000 (0%)]	Loss: 2.300511	Acc: 13.281%
Epoch: 0 [128/50000 (0%)]	Loss: 2.401274	Acc: 10.156%
Epoch: 0 [256/50000 (1%)]	Loss: 2.299637	Acc: 10.938%
Epoch: 0 [384/50000 (1%)]	Loss: 2.331532	Acc: 9.961%
Epoch: 0 [512/50000 (1%)]	Loss: 2.333029	Acc: 10.000%
Epoch: 0 [640/50000 (1%)]	Loss: 2.318985	Acc: 10.026%
Epoch: 0 [768/50000 (2%)]	Loss: 2.302854	Acc: 9.710%
Epoch: 0 [896/50000 (2%)]	Loss: 2.304626	Acc: 9.570%
Epoch: 0 [1024/50000 (2%)]	Loss: 2.309824	Acc: 9.722%
Epoch: 0 [1152/50000 (2%)]	Loss: 2.290468	Acc: 10.391%
Epoch: 0 [1280/50000 (3%)]	Loss: 2.300295	Acc: 10.227%
Epoch: 0 [1408/50000 (3%)]	Loss: 2.322746	Acc: 9.896%
Epoch: 0 [1536/50000 (3%)]	Loss: 2.290113	Acc: 10.036%
Epoch: 0 [1664/50000 (3%)]	Loss: 2.294221	Acc: 10.156%
Epoch: 0 [1792/50000 (4%)]	Loss: 2.304645	Acc: 10.312%
Epoch: 0 [1920/50000 (4%)]	Loss: 2.293050	Acc: 10.498%
Epoch: 0 [2048/50000 (4%)]	Loss: 2.300561	Acc: 10.156%
Epoch: 0 [2176/50000 (4%)]	Loss: 2.276689	Acc: 10.156%
Epoch: 0 [2304/50000 (5%)

In [ ]:
## 打开终端，输入 tensorboard --logdir=runs
## 然后在浏览器中输入 http://localhost:6006/ 就可以看到 tensorboard 的界面了
